In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
!apt-get install p7zip
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/train.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/test.tsv.7z
!p7zip -d -f -k /kaggle/input/mercari-price-suggestion-challenge/sample_submission.csv.7z

In [ ]:
!unzip /kaggle/input/mercari-price-suggestion-challenge/sample_submission_stg2.csv.zip
!unzip /kaggle/input/mercari-price-suggestion-challenge/test_stg2.tsv.zip

In [ ]:
train_data = pd.read_csv('train.tsv', sep = '\t')
test_data = pd.read_csv('test_stg2.tsv', sep='\t')

In [ ]:
train_data

In [ ]:
train_data.info()

In [ ]:
train_data.brand_name.fillna(value = "missing", inplace = True)
test_data.brand_name.fillna(value = "missing", inplace = True)

train_data.category_name.fillna(value = "Other/Other/Other", inplace = True)
test_data.category_name.fillna(value = "Other/Other/Other", inplace = True)

train_data.item_description.fillna(value = "No description yet", inplace = True)
test_data.item_description.fillna(value = "No description yet", inplace = True)

## Pre-Processing:
>   Fix missing data    

In [ ]:
full_data = pd.concat([train_data, test_data], sort=False)

In [ ]:
full_data.isnull().sum()

In [ ]:
train_data.item_description.fillna(value = "No description yet", inplace = True)
test_data.item_description.fillna(value = "No description yet", inplace = True)

train_data.category_name.fillna(value = "Other/Other/Other", inplace = True)
test_data.category_name.fillna(value = "Other/Other/Other", inplace = True)

train_data.brand_name.fillna(value = "missing", inplace = True)
test_data.brand_name.fillna(value = "missing", inplace = True)

In [ ]:
train_data['general_category']='' 
train_data['subcategory_1'] = '' 
train_data['subcategory_2'] = ''

test_data['general_category']='' 
test_data['subcategory_1'] = '' 
test_data['subcategory_2'] = ''

In [ ]:
def split(txt):
    try :
        return txt.split("/")
    except :
        return ("missing", "missing", "missing")

In [ ]:
train_data['general_category'],train_data['subcategory_1'],train_data['subcategory_2'] = \
zip(*train_data['category_name'].apply(lambda x: split(x)))

test_data['general_category'],test_data['subcategory_1'],test_data['subcategory_2'] = \
zip(*test_data['category_name'].apply(lambda x: split(x)))

In [ ]:
train_data

In [ ]:
train_data.price.plot.hist(bins=50, figsize=(8,4), edgecolor='white',range=[0,300])
plt.title('Price Distribution')

In [ ]:
train_data.hist(bins=50, figsize=(20,15))
plt.show()

In [ ]:
train_data['item_condition_id'].value_counts().plot(kind='bar')

In [ ]:
train_data['general_category'].value_counts().plot(kind='bar')

In [ ]:
train_data['subcategory_1'].value_counts().plot(kind='bar')
train_data['subcategory_1'].value_counts()

In [ ]:
train_data['subcategory_2'].value_counts().plot(kind='bar')
train_data['subcategory_2'].value_counts()

In [ ]:
log_prices = np.log1p(train_data.price)
log_prices.hist()

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train_item_condition_id = scaler.fit_transform(train_data['item_condition_id'].values.reshape(-1,1))

In [ ]:
from sklearn.preprocessing import LabelBinarizer

lb_brand_name = LabelBinarizer(sparse_output=True)
X_train_brand = lb_brand_name.fit_transform(train_data['brand_name'])

lb_shipping = LabelBinarizer(sparse_output=True)
X_train_shipping = lb_shipping.fit_transform(train_data['shipping'])

lb_cat_0 = LabelBinarizer(sparse_output=True)
X_train_general_category = lb_cat_0.fit_transform(train_data['general_category'])

lb_cat_1 = LabelBinarizer(sparse_output=True)
X_train_subcategory_1 = lb_cat_1.fit_transform(train_data['subcategory_1'])

lb_cat_2 = LabelBinarizer(sparse_output=True)
X_train_subcategory_2 = lb_cat_2.fit_transform(train_data['subcategory_2'])

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

tfidf_desc = TfidfVectorizer(max_features=50000, ngram_range=(1, 1), stop_words='english', norm='l2',lowercase=True)
name_vectorizer = CountVectorizer(stop_words='english')

X_train_descp = tfidf_desc.fit_transform(train_data['item_description'])
X_train_name = name_vectorizer.fit_transform(train_data['name'])

In [ ]:
corr_matrix = train_data.corr()
corr_matrix

In [ ]:
from scipy.sparse import hstack

X_train = hstack((X_train_name, X_train_descp, X_train_brand, X_train_item_condition_id, X_train_shipping, X_train_general_category, X_train_subcategory_1, X_train_subcategory_2)).tocsr()

In [ ]:
import seaborn as sns

sns.heatmap(corr_matrix, 
            xticklabels=corr_matrix.columns.values,
            yticklabels=corr_matrix.columns.values)

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.1, random_state=42)
for train_index, test_index in split.split(train_data, train_data['general_category']):
    train_idx = train_index
    test_idx = test_index

In [ ]:
test = X_train[test_idx]
log_test_prices = log_prices.iloc[test_idx]
train = X_train[train_idx]
log_train_prices = log_prices.iloc[train_idx]

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

def performance(ytrue, ypredicted):
    print("MSE: ",np.sqrt(mean_squared_error(ytrue, ypredicted))) 
    print("MAE: ", np.sqrt(mean_absolute_error(ytrue, ypredicted))) 

In [ ]:
## Random Forest
from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(verbose=True, max_depth=15, n_estimators=30)
forest_reg.fit(train, log_train_prices)

performance(log_test_prices, forest_reg.predict(test))

In [ ]:
## SGD Regressor

from sklearn.linear_model import SGDRegressor

# SkLearn SGD classifier
clf_ = SGDRegressor()
clf_.fit(train, log_train_prices)
performance(log_test_prices, clf_.predict(test))